## 1. Genetický algoritmus

Tento kód implementuje genetický algoritmus (GA) na optimalizáciu stratégií pre hru Blackjack. Cieľom je nájsť optimálnu stratégiu rozhodovania sa (**hit/stand**), ktorá maximalizuje očakávaný výnos hráča.

### Parametre genetického algoritmu
- **population_size** (predvolená hodnota: 50): Veľkosť populácie (počet stratégií v každej generácii)
- **mutation_rate** (predvolená hodnota: 0.01): Pravdepodobnosť mutácie každého rozhodnutia v stratégii
- **generations** (predvolená hodnota: 100): Počet generácií pre GA
- **elite_count** (nastavené na 2): Počet najlepších jedincov, ktorí sú zachovaní bez zmien do ďalšej generácie

### Inicializácia matíc

Algoritmus inicializuje populáciu stratégií pre Blackjack, kde každá stratégia pozostáva z dvoch matíc:

- **Matica hard_totals (17x10):**
    - Riadky predstavujú súčet hráčových kariet (4-20)
    - Stĺpce predstavujú kartu dealera (2-10, A)
    - Hodnoty 0/1 označujú akciu (0=stand, 1=hit)

- **Matica soft_totals (9x10):**
    - Riadky predstavujú súčet hráčových kariet s použiteľným esom (12-20)
    - Stĺpce predstavujú kartu dealera (2-10, A)
    - Hodnoty 0/1 označujú akciu (0=stand, 1=hit)

Inicializácia prebieha v metóde `create_blackjack_matrices()`, kde sa pre každú pozíciu v maticiach náhodne vyberie hodnota 0 alebo 1 (stand alebo hit).

### Evolučný proces

Genetický algoritmus iteratívne vylepšuje stratégie v priebehu generácií:

1. **Vyhodnotenie fitness:**
     - Každá stratégia je vyhodnotená simuláciou 20 000 hier Blackjacku.
     - Fitness je priemerná výhra/prehra naprieč týmito hrami.
     - Vyhodnotenie prebieha paralelne pomocou `ProcessPoolExecutor`.

2. **Selekcia:**
     - Vyberie sa najlepšia polovica stratégií podľa ich fitness hodnoty.
     - Tieto stratégie sa stanú rodičmi pre novú generáciu.

3. **Elitizmus:**
     - Najlepšie 2 stratégie z aktuálnej generácie sú zachované bez zmeny.
     - To zabezpečuje, že kvalita populácie neklesne.

4. **Kríženie:**
     - Nová stratégia vzniká kombináciou dvoch rodičovských stratégií.
     - Pre každú pozíciu v matici je 50% šanca, že hodnota bude zdedená od prvého rodiča, inak od druhého.
     - Kríženie sa vykonáva samostatne pre `hard_totals` aj `soft_totals`.

5. **Mutácia:**
     - S malou pravdepodobnosťou (`mutation_rate`, defaultne 0.01) sa hodnoty v maticiach náhodne zmenia.
     - Mutácia prebehne inverziou hodnoty (0→1 alebo 1→0).
     - Umožňuje algoritmu skúmať nové možnosti mimo aktuálnej populácie.

Algoritmus sleduje najlepšiu stratégiu nájdenú počas celého evolučného procesu a jej fitness hodnotu. Po dokončení všetkých generácií vráti túto najlepšiu stratégiu spolu s históriou fitness pre analýzu konvergencie algoritmu.

### Cieľ experimentu
Zistiť, za akých podmienok sa genetický algoritmus dokáže najviac priblížiť k optimálnej stratégii.

### Merané metriky
- Priemerná výhra oboch stratégií pri simulácii hier
- Percentuálna zhoda výstupnej stratégie s optimálnou stratégiou
- Konvergenčná rýchlosť pri rôznych nastaveniach parametrov

### Parametre na experimentovanie
Budeme skúmať, ako nasledujúce parametre ovplyvňujú kvalitu výslednej stratégie:

#### Veľkosť populácie (`population_size`):
- **Menšie hodnoty (10, 20):** Rýchlejší výpočet, ale menej diverzity v populácii
- **Väčšie hodnoty (50, 100):** Pomalší výpočet, ale potenciálne lepšie preskúmanie priestoru riešení

#### Miera mutácie (`mutation_rate`):
- **Nízke hodnoty (0.001, 0.005):** Pomalšia evolúcia, ale stabilnejšia konvergencia
- **Stredné hodnoty (0.01, 0.02):** Vyvážená explorácia a exploitácia
- **Vysoké hodnoty (0.05, 0.1):** Agresívnejšia explorácia, ale riziko nestability

#### Počet generácií (`generations`):
- Rôzne hodnoty (50, 100, 200, 500) na sledovanie, po koľkých generáciách dochádza k saturácii výkonnosti

#### `Elite_count`:
- Rôzne hodnoty (1, 2, 5, 10) pre posúdenie vplyvu elitizmu na kvalitu výslednej stratégie


In [ ]:
from genetic_algorithm import BlackjackGA
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import numpy as np
from evaluate_utils import evaluate_strategy, optimal_hard_totals, optimal_soft_totals

# Konfigurácie experimentu
population_sizes = [10, 20]
mutation_rates = [0.01, 0.05]
generations_list = [50, 100]
episode_counts = [10000, 20000]

optimal_reward_10000 = evaluate_strategy((optimal_hard_totals, optimal_soft_totals), episodes=10000)
optimal_reward_20000 = evaluate_strategy((optimal_hard_totals, optimal_soft_totals), episodes=20000)

# Príprava dataframe pre výsledky
results = []

# Experimentovanie s rôznymi parametrami
for pop_size in tqdm(population_sizes, desc="Population Sizes"):
    for mutation in mutation_rates:
        for gens in generations_list:
            for episode_count in episode_counts:
                # Spustenie algoritmu s aktuálnymi parametrami
                solver = BlackjackGA(population_size=pop_size, mutation_rate=mutation, generations=gens, episodes=episode_count)
                if episode_count == 10000:
                    optimal_reward = optimal_reward_10000
                else:
                    optimal_reward = optimal_reward_20000

                best_strategy, best_fitness, fitness_history = solver.run()
                
                # Vyhodnotenie výsledkov
                output_reward = evaluate_strategy(best_strategy, episodes=episode_count)
                
                # Výpočet rozdielu medzi výstupnou odmenou a optimálnou odmenou
                reward_difference = optimal_reward - output_reward

                # Uloženie výsledkov
                results.append({
                    'population_size': pop_size,
                    'mutation_rate': mutation,
                    'generations': gens,
                    'episodes': episode_count,
                    'output_reward': output_reward,
                    'optimal_reward': optimal_reward,
                    'reward_difference': reward_difference
                })

            

# Konverzia na dataframe
results_df = pd.DataFrame(results)

# Zobrazenie výsledkov
print(results_df.head())

# Vizualizácia vplyvu parametrov na zhodu s optimálnou stratégiou
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
sns.boxplot(x='population_size', y='reward_difference', data=results_df)
plt.title('Vplyv veľkosti populácie na zhodu s optimálnou stratégiou')

plt.subplot(2, 2, 2)
sns.boxplot(x='mutation_rate', y='reward_difference', data=results_df)
plt.title('Vplyv miery mutácie na zhodu s optimálnou stratégiou')

plt.subplot(2, 2, 3)
sns.boxplot(x='generations', y='reward_difference', data=results_df)
plt.title('Vplyv počtu generácií na zhodu s optimálnou stratégiou')

plt.subplot(2, 2, 4)
sns.boxplot(x='episode_count', y='reward_difference', data=results_df)
plt.title('Vplyv počtu epizod na zhodu s optimálnou stratégiou')

plt.tight_layout()
plt.show()

# Vytvorenie tepelnej mapy rozdielov medzi najlepšou nájdenou stratégiou a optimálnou
best_result_idx = results_df['reward_difference'].idxmax()
best_params = results_df.iloc[best_result_idx]

print(f"Best parameters: population_size={best_params['population_size']}, " +
      f"mutation_rate={best_params['mutation_rate']}, generations={best_params['generations']}, " +
      f"elite_count={best_params['elite_count']}")

# Spustíme algoritmus s najlepšími parametrami pre vizualizáciu
best_solver = BlackjackGA(
    population_size=int(best_params['population_size']), 
    mutation_rate=best_params['mutation_rate'], 
    generations=int(best_params['generations']),
    episodes=int(best_params['episode_count'])
)
best_strategy, fitness, fitness_history = best_solver.run()

# Vizualizácia histórie fitness
plt.figure(figsize=(10, 6))
plt.plot(fitness_history, label='Fitness History', color='blue')
plt.axhline(y=best_fitness, color='red', linestyle='--', label='Best Fitness')
plt.title('Fitness History Across Generations')
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.legend()
plt.grid()
plt.show()

Optimal reward: -0.0603


Population Sizes:   0%|          | 0/2 [00:00<?, ?it/s]

Generation 1: Best Fitness = -0.2907
Generation 2: Best Fitness = -0.2645
Generation 3: Best Fitness = -0.2744
Generation 4: Best Fitness = -0.258
Generation 5: Best Fitness = -0.2454
Generation 6: Best Fitness = -0.23
Generation 7: Best Fitness = -0.2204
Generation 8: Best Fitness = -0.1947
Generation 9: Best Fitness = -0.2028
Generation 10: Best Fitness = -0.197
Generation 11: Best Fitness = -0.2009
Generation 12: Best Fitness = -0.186
Generation 13: Best Fitness = -0.1988
Generation 14: Best Fitness = -0.1879
Generation 15: Best Fitness = -0.1752
Generation 16: Best Fitness = -0.1844
Generation 17: Best Fitness = -0.1756
Generation 18: Best Fitness = -0.1689
Generation 19: Best Fitness = -0.1632
Generation 20: Best Fitness = -0.1762
Generation 21: Best Fitness = -0.1698
Generation 22: Best Fitness = -0.172
Generation 23: Best Fitness = -0.1598
Generation 24: Best Fitness = -0.1577
Generation 25: Best Fitness = -0.1562
Generation 26: Best Fitness = -0.1518
Generation 27: Best Fitness

Process ForkProcess-2037:
Process ForkProcess-2038:
Process ForkProcess-2039:
Process ForkProcess-2036:
Process ForkProcess-2035:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multipr

# 2. Neurónová sieť

Tento kód implementuje model neurónovej siete, ktorá sa učí optimálnu stratégiu pre hru Blackjack. Model je trénovaný pomocou supervised learning na vopred definovanej optimálnej stratégii a následne je schopný predpovedať akcie (hit/stand) pre rôzne herné stavy.

## Architektúra siete

```python
model = keras.Sequential([
    layers.Input(shape=(3,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
```

Architektúra pozostáva z:

- **Vstupnej vrstvy** pre 3 parametre:
  - skóre hráča
  - karta dealera
  - použiteľné eso
- **Dvoch skrytých vrstiev**:
  - 64 neurónov s aktivačnou funkciou **ReLU**
  - 32 neurónov s aktivačnou funkciou **ReLU**
- **Výstupnej vrstvy** s aktivačnou funkciou **sigmoid**, ktorá určuje pravdepodobnosť akcie:
  - `1 = hit`, `0 = stand`

---

## Trénovacie dáta

Trénovacie dáta sú generované funkciou `create_training_data()`, ktorá vytvára páry **(stav, akcia)** podľa základnej stratégie Blackjacku.

### Kombinácie:

- **Skóre hráča**: 4–21  
- **Karta dealera**: 1–10 (kde `1 = Eso`, `10 = 10/J/Q/K`)  
- **Stav esa**: použiteľné / nepoužiteľné

### Normalizácia vstupov (na rozsah [0,1]):

- Skóre hráča: `(player_sum - 4) / 17.0`  
- Karta dealera: `(dealer_card - 1) / 9.0`  
- Použiteľné eso: `1` alebo `0`

### Určenie akcií:

- Podľa základnej stratégie pre:
  - **Tvrdé súčty** (bez použiteľného esa)
  - **Mäkké súčty** (s použiteľným esom)

---

## Parametre trénovania

Trénovanie prebieha v metóde `train()` s nasledujúcimi parametrami:

- `epochs` *(predvolene 100)*: počet trénovacích epoch  
- `batch_size` *(predvolene 16)*: veľkosť dávky pre trénovanie  
- **Validácia** na 10 % trénovacích dát  
- **Optimalizátor**: `Adam`  
- **Loss funkcia**: `binary_crossentropy`  
- **Metrika**: `accuracy`

---

## Vyhodnocovanie modelu

Model je vyhodnocovaný metódou `evaluate()`, ktorá simuluje hru Blackjack pomocou knižnice **Gymnasium**:

- Prebehne `num_episodes` hier *(predvolene 20 000)*
- V každej hre:
  - Model určuje akciu (`hit` / `stand`) podľa aktuálneho stavu
  - Zaznamenáva sa priemerna odmena




In [ ]:
from neural_network import BlackjackNN, optimal_hard_totals, optimal_soft_totals, evaluate_strategy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
    
layer_configs = [
    [(32, 'relu'), (16, 'relu')],
    [(64, 'relu'), (32, 'relu')],
    [(128, 'relu'), (64, 'relu')]
]

optimal_reward = evaluate_strategy((optimal_hard_totals, optimal_soft_totals))

for config in layer_configs:
    print(f"\nTraining with layer configuration: {config}")
    learner = BlackjackNN(layer_config=config)
    history = learner.train(epochs=100, batch_size=32)
    
    model_reward = learner.evaluate()
    
    # Save results
    config_name = '-'.join([f"{units}" for units, _ in config])
    results.append({
        'config': config_name,
        'optimal_reward': optimal_reward,
        'model_reward': model_reward,
        'reward_difference': abs(optimal_reward) - abs(model_reward),
        'accuracy': max(history.history['val_accuracy']),
        'num_layers': len(config),
        'total_neurons': sum([units for units, _ in config])
    })
    
    print(f"Optimal Strategy Avg Reward: {optimal_reward:.4f}")
    print(f"Model Strategy Avg Reward: {model_reward:.4f}")
    print(f"Difference: {optimal_reward - model_reward:.4f}")
    print(f"Validation Accuracy: {max(history.history['val_accuracy']):.4f}")

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Create visualizations
plt.figure(figsize=(15, 10))

# Plot reward comparison
plt.subplot(2, 2, 1)
sns.barplot(x='config', y='model_reward', data=results_df, color='blue', label='Model')
sns.barplot(x='config', y='optimal_reward', data=results_df, color='green', alpha=0.5, label='Optimal')
plt.title('Model vs Optimal Strategy Reward')
plt.xlabel('Network Configuration (neurons per layer)')
plt.ylabel('Average Reward')
plt.legend()

# Plot reward difference
plt.subplot(2, 2, 2)
sns.barplot(x='config', y='reward_difference', data=results_df)
plt.title('Reward Difference (Optimal - Model)')
plt.xlabel('Network Configuration (neurons per layer)')
plt.ylabel('Difference')

# Plot accuracy vs. network size
plt.subplot(2, 2, 3)
sns.scatterplot(x='total_neurons', y='accuracy', size='num_layers', data=results_df, sizes=(100, 200))
plt.title('Accuracy vs Network Size')
plt.xlabel('Total Neurons')
plt.ylabel('Validation Accuracy')

# Plot accuracy vs. reward difference
plt.subplot(2, 2, 4)
sns.scatterplot(x='accuracy', y='reward_difference', hue='config', data=results_df, s=100)
plt.title('Accuracy vs Reward Difference')
plt.xlabel('Validation Accuracy')
plt.ylabel('Reward Difference (Optimal - Model)')

plt.tight_layout()
plt.savefig('network_configurations_comparison.png')
plt.show()

# Print summary
print("\nSummary of Network Configurations:")
summary = results_df[['config', 'model_reward', 'optimal_reward', 'reward_difference', 'accuracy']]
summary = summary.sort_values('reward_difference')
print(summary.to_string(index=False, float_format=lambda x: f"{x:.4f}"))


# # Plot training history
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['loss'], label='Training Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Loss During Training')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
    
# plt.subplot(1, 2, 2)
# plt.plot(history.history['accuracy'], label='Training Accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
# plt.title('Accuracy During Training')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
    
# plt.tight_layout()
# plt.savefig('training_history.png')
# plt.show()
    

    

2025-05-11 12:25:01.338930: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-11 12:25:01.340517: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-11 12:25:01.349499: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-11 12:25:01.375459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-11 12:25:01.403503: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 


Training with layer configuration: [(32, 'relu'), (16, 'relu')]


I0000 00:00:1746959106.922667 2332104 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-11 12:25:06.923350: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


State: [0.0, 0.0, 0], Action: 1
State: [0.0, 0.1111111111111111, 0], Action: 1
State: [0.0, 0.2222222222222222, 0], Action: 1
State: [0.0, 0.3333333333333333, 0], Action: 1
State: [0.0, 0.4444444444444444, 0], Action: 1
State: [0.0, 0.5555555555555556, 0], Action: 1
State: [0.0, 0.6666666666666666, 0], Action: 1
State: [0.0, 0.7777777777777778, 0], Action: 1
State: [0.0, 0.8888888888888888, 0], Action: 1
State: [0.0, 1.0, 0], Action: 1
State: [0.058823529411764705, 0.0, 0], Action: 1
State: [0.058823529411764705, 0.1111111111111111, 0], Action: 1
State: [0.058823529411764705, 0.2222222222222222, 0], Action: 1
State: [0.058823529411764705, 0.3333333333333333, 0], Action: 1
State: [0.058823529411764705, 0.4444444444444444, 0], Action: 1
State: [0.058823529411764705, 0.5555555555555556, 0], Action: 1
State: [0.058823529411764705, 0.6666666666666666, 0], Action: 1
State: [0.058823529411764705, 0.7777777777777778, 0], Action: 1
State: [0.058823529411764705, 0.8888888888888888, 0], Action: 1

KeyboardInterrupt: 